In [13]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.io as scio
from scipy.optimize import fsolve
from scipy.optimize import root

In [14]:
# 导入基准情形的变量和参数
equilibrium2010 = np.load('./data/equilibrium2010.npz')
R = equilibrium2010['R']
X = equilibrium2010['X']
m = equilibrium2010['m']
pi = equilibrium2010['pi']
L = equilibrium2010['L']
L_bar = equilibrium2010['L_bar']
K = equilibrium2010['K']

parameter = np.load('./data/Parameter.npz')
kappa = parameter['kappa']
theta = parameter['theta']
beta = parameter['beta']
psi_ag = parameter['psi_ag']
psi_na = parameter['psi_na']
alpha_ag = parameter['alpha_ag']
alpha_na = parameter['alpha_na']
sigma_ag = parameter['sigma_ag']
sigma_na = parameter['sigma_na']
eta_ag_ag = parameter['eta_ag_ag']
eta_ag_na = parameter['eta_ag_na']
eta_na_ag = parameter['eta_na_ag']
eta_na_na = parameter['eta_na_na']
phiu = parameter['phiu']
phip = parameter['phip']
chi = parameter['chi']
eta = parameter['eta']
delta = parameter['delta']
xi_ag = parameter['xi_ag']
xi_na = parameter['xi_na']
t = parameter['t']

In [15]:
# 导入外生变量的反事实变化

## 保持各外生变量均不变
dKp = np.ones([60, 1])
dKh = np.ones([60, 1])
dT = np.ones([60, 1])
dtau = np.ones([60, 60])
dmu = np.ones([60, 60])

In [16]:
# 定义求解反事实均衡的函数

def SolveCounter(XX):
    dw = XX[:60].reshape([60, 1])
    dL = XX[60:120].reshape([60, 1])
    dp = XX[120:].reshape([60, 1])

    ## (a)价格指数的变化
    dpdp_wp_c = np.repeat((dp.reshape([30, 2]) ** np.array([psi_ag * beta, psi_na * beta]).reshape([1, 2])).prod(axis=1).reshape([30, 1]), 2, axis=1).reshape([60, 1])
    drh = dw * dL / dKh
    dP =  dpdp_wp_c *  (drh ** (1 - beta))

    ## (b)公共支出的变化
    dG = dw * dL / dP

    ## (c)单位投入束成本的相对变化
    dw_wp = (dw.reshape([30, 2]) ** np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    drp = dw * dL / dKp
    drp_wp = (drp.reshape([30, 2]) ** np.array([sigma_ag, sigma_na]).reshape([1, 2])).reshape([60, 1])
    dpdp_wp_p_ag = (dp.reshape([30, 2]) ** np.array([eta_ag_ag, eta_na_ag]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p_na = (dp.reshape([30, 2]) ** np.array([eta_ag_na, eta_na_na]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p = np.concatenate([dpdp_wp_p_ag, dpdp_wp_p_na], axis=1).reshape([60, 1])
    dc = dw_wp * drp_wp * dpdp_wp_p

    ## (d)贸易份额的相对变化
    dpi_num = dT * ((dtau * dc) ** (-theta)) * (dG ** (phip * theta)) * (dL ** (theta * delta)) # 分子(考虑生产函数中存在规模经济效应)
    dpi_den = (dpi_num * pi).sum(axis=0).reshape([60, 1]) # 分母
    dpi = (dpi_num.T / dpi_den).T
    pi_prime = pi * dpi # 反事实情形下的贸易份额

    ## (e)最终品价格的相对变化
    dp1 = dpi_den ** (-1 / theta)

    ## (f)收入倍率的相对变化
    wL = (R.reshape([30, 2]) * np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1]) # 计算总工资收入
    w = wL / L # 名义工资收入
    w_prime = w * dw
    coef_rK_ag = (alpha_ag + sigma_ag - (beta + t - beta * t) * alpha_ag) / ((beta + t - beta * t) * alpha_ag)
    coef_rK_na = sigma_na / alpha_na + (1 - beta) * (1 - t)
    rKrK = (wL.reshape([30, 2]) * np.array([coef_rK_ag, coef_rK_na]).reshape([1, 2])).reshape([60, 1])  # 计算出城市部门和农村部门的土地总收入
    rho_prime = rKrK * dw * dL / L_bar # 土地改革后，户籍居民的人均土地收入
    rho_prime[1::2] = 0 # 城市部门居民没有土地收入，所以将偶数行替换为0
    delta_prime = 1 + (w_prime ** (-1)) @ rho_prime.T # 改革后的收入倍率
    delta_diag = ((L / (L_bar * np.diag(m).reshape([60, 1]))).reshape([30, 2]) * np.array([coef_rK_ag, 0]).reshape([1, 2]) + 1).reshape([60, 1]) # 计算delta的主对角线元素
    ddelta = delta_prime # ddelta和delta_prime的非主对角线元素是一样的
    np.fill_diagonal(ddelta, np.diag(delta_prime).reshape([60, 1]) / delta_diag) # 计算并替换ddelta的主对角线元素

    ## (g)流动份额的相对变化
    dg = dG / (dL ** chi) # 人均公共支出的相对变化
    dW = dw / dP # 实际工资的相对变化
    dm_num = (ddelta * (dmu ** (-1)) * (dg ** phiu) * dW) ** kappa # 分子
    dm_den = (dm_num * m).sum(axis=0).reshape([60, 1]) # 分母
    dm = (dm_num.T / dm_den).T
    m_prime = m * dm

    ## (h)常住人口数的相对变化
    dL1 = (m_prime @ L_bar) / L

    ## (i)计算反事实下的总产出与总支出
    R_prime = R * dw * dL
    wL_prime = wL * dw * dL
    X_prime = np.linalg.inv(pi_prime) @ R_prime ## 根据总产出R_prime计算出的总支出X_prime
    vL_prime = (wL_prime.reshape([30, 2]) * np.array([(alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1]).reshape([1, 2])).reshape([60, 1])
    PG_prime = (wL_prime.reshape([30, 2]) * np.array([t * (alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1 - beta + sigma_na / alpha_na + beta * t]).reshape([1, 2])).reshape([60, 1])
    A_prime = (beta * (1 - t) * np.repeat(vL_prime.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([psi_ag, psi_na]).reshape([1, 2])).reshape([60, 1])
    C_prime = (np.repeat(PG_prime.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([xi_ag, xi_na]).reshape([1, 2])).reshape([60, 1])
    B_ag_prime = ((R_prime.reshape([30, 2]) * np.array([eta_ag_ag, eta_ag_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B_na_prime = ((R_prime.reshape([30, 2]) * np.array([eta_na_ag, eta_na_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B_prime = np.concatenate([B_ag_prime, B_na_prime], axis=1).reshape([60, 1])
    X_prime1 = A_prime + B_prime + C_prime ## 根据各项支出计算出的总支出X_prime1，均衡下应该与前面的X_prime相等

    ## (j)均衡条件
    res1 = dp1 - dp
    res2 = dL1 - dL
    res3 = X_prime1 - X_prime

    return np.concatenate([res1, res2, res3]).flatten().astype(float)

In [17]:
# 定义储存反事实变量的函数
def counterfautual_result(dw, dL, dp):
    dpdp_wp_c = np.repeat((dp.reshape([30, 2]) ** np.array([psi_ag * beta, psi_na * beta]).reshape([1, 2])).prod(axis=1).reshape([30, 1]), 2, axis=1).reshape([60, 1])
    drh = dw * dL / dKh
    dP =  dpdp_wp_c *  (drh ** (1 - beta))
    dG = dw * dL / dP
    dw_wp = (dw.reshape([30, 2]) ** np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    drp = dw * dL / dKp
    drp_wp = (drp.reshape([30, 2]) ** np.array([sigma_ag, sigma_na]).reshape([1, 2])).reshape([60, 1])
    dpdp_wp_p_ag = (dp.reshape([30, 2]) ** np.array([eta_ag_ag, eta_na_ag]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p_na = (dp.reshape([30, 2]) ** np.array([eta_ag_na, eta_na_na]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p = np.concatenate([dpdp_wp_p_ag, dpdp_wp_p_na], axis=1).reshape([60, 1])
    dc = dw_wp * drp_wp * dpdp_wp_p
    dpi_num = dT * ((dtau * dc) ** (-theta)) * (dG ** (phip * theta)) * (dL ** (theta * delta)) # 分子(考虑生产函数中存在规模经济效应)
    dpi_den = (dpi_num * pi).sum(axis=0).reshape([60, 1]) # 分母
    dpi = (dpi_num.T / dpi_den).T
    pi_prime = pi * dpi # 反事实情形下的贸易份额
    wL = (R.reshape([30, 2]) * np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1]) # 计算总工资收入
    w = wL / L # 名义工资收入
    w_prime = w * dw
    coef_rK_ag = (alpha_ag + sigma_ag - (beta + t - beta * t) * alpha_ag) / ((beta + t - beta * t) * alpha_ag)
    coef_rK_na = sigma_na / alpha_na + (1 - beta) * (1 - t)
    rKrK = (wL.reshape([30, 2]) * np.array([coef_rK_ag, coef_rK_na]).reshape([1, 2])).reshape([60, 1])  # 计算出城市部门和农村部门的土地总收入
    rho_prime = rKrK * dw * dL / L_bar # 土地改革后，户籍居民的人均土地收入
    rho_prime[1::2] = 0 # 城市部门居民没有土地收入，所以将偶数行替换为0
    delta_prime = 1 + (w_prime ** (-1)) @ rho_prime.T # 改革后的收入倍率
    delta_diag = ((L / (L_bar * np.diag(m).reshape([60, 1]))).reshape([30, 2]) * np.array([coef_rK_ag, 0]).reshape([1, 2]) + 1).reshape([60, 1]) # 计算delta的主对角线元素
    ddelta = delta_prime # ddelta和delta_prime的非主对角线元素是一样的
    np.fill_diagonal(ddelta, np.diag(delta_prime).reshape([60, 1]) / delta_diag) # 计算并替换ddelta的主对角线元素
    dg = dG / (dL ** chi) # 人均公共支出的相对变化
    dW = dw / dP # 实际工资的相对变化
    dm_num = (ddelta * (dmu ** (-1)) * (dg ** phiu) * dW) ** kappa # 分子
    dm_den = (dm_num * m).sum(axis=0).reshape([60, 1]) # 分母
    dm = (dm_num.T / dm_den).T
    m_prime = m * dm
    R_prime = R * dw * dL
    X_prime = np.linalg.inv(pi_prime) @ R_prime ## 根据总产出R_prime计算出的总支出X_prime

    counterfautual_dict = {
        'dw' : dw,
        'dL' : dL,
        'dp' : dp,
        'dG' : dG,
        'dc' : dc,
        'dpi' : dpi,
        'pi_prime' : pi_prime,
        'dP' : dP,
        'ddelta' : ddelta,
        'dW' : dW,
        'dmu' : dmu,
        'dm' : dm,
        'm_prime' : m_prime,
        'R_prime' : R_prime,
        'X_prime' : X_prime,
        'drh' : drh
    }

    return counterfautual_dict

In [18]:
# 求解反事实均衡
XX = root(SolveCounter, x0=np.ones(3*60), method='lm') # 求解非线性方程组
dw = XX.x[:60].reshape([60, 1])
dL = XX.x[60:120].reshape([60, 1])
dp = XX.x[120:].reshape([60, 1])

counterfautual_dict = counterfautual_result(dw, dL, dp)
np.savez('./data/Counterfactuals2010.npz', **counterfautual_dict)